<a href="https://colab.research.google.com/github/SaqlainHussainShah/GenAI/blob/main/BERT_model_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
finetuning_dataset = "/content/drive/My Drive/shaharyar_bert/finetuning_dataset.csv"


In [ ]:
import pandas as pd

In [ ]:
result_df = pd.read_csv(finetuning_dataset)

In [ ]:
result_df.head()

,CVE_ID,Input_Combined_Description,Output_Combined_Description
0,CVE-2016-0110,Microsoft Internet Explorer 10 through 11 and ...,Adversaries may send phishing messages to gain...
1,CVE-2016-0111,Microsoft Internet Explorer 9 through 11 and M...,Adversaries may gain access to a system throug...
2,CVE-2016-0112,Microsoft Internet Explorer 9 through 11 allow...,Adversaries may gain access to a system throug...
3,CVE-2016-0113,Microsoft Internet Explorer 9 through 11 allow...,Adversaries may exploit software vulnerabiliti...
4,CVE-2016-0114,Microsoft Internet Explorer 11 allows remote a...,Adversaries may gain access to a system throug...


In [ ]:
import pandas as pd
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW

# Define custom dataset class
class Seq2SeqDataset(Dataset):
    def __init__(self, input_sentences, output_sentences, tokenizer, max_length):
        self.input_sentences = input_sentences
        self.output_sentences = output_sentences
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_sentences)

    def __getitem__(self, idx):
        input_text = self.input_sentences[idx]
        output_text = self.output_sentences[idx]

        input_encodings = self.tokenizer(input_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        output_encodings = self.tokenizer(output_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')

        return {
            'input_ids': input_encodings['input_ids'].flatten(),
            'attention_mask': input_encodings['attention_mask'].flatten(),
            'labels': output_encodings['input_ids'].flatten(),
        }

# Load your DataFrame
# result_df = pd.read_csv('path_to_your_dataframe.csv')

# Assuming your DataFrame 'result_df' contains 'Input Combined Description' and 'Output Combined Description' columns
input_sentences = result_df['Input_Combined_Description'].tolist()
output_sentences = result_df['Output_Combined_Description'].tolist()

# Define tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base').to('cuda')

# Define dataset and dataloader
dataset = Seq2SeqDataset(input_sentences, output_sentences, tokenizer, max_length=64)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Fine-tuning loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss}")

    # Optionally, adjust learning rate
    scheduler.step()

# Save the fine-tuned model
model.save_pretrained("fine_tuned_bart_model")


NameError: name 'result_df' is not defined

In [ ]:
model.save_pretrained("/content/drive/My Drive/shaharyar_bert/fine_tuned_bart_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [ ]:
import torch
import pickle

# Save the fine-tuned model
torch.save(model, '/content/drive/My Drive/shaharyar_bert/fine_tuned_bart_model.pkl')


In [ ]:
# Load the model from the pickle file
model = torch.load('fine_tuned_bart_model.pkl')


In [ ]:
techniques_df = pd.read_csv('/content/drive/My Drive/shaharyar_bert/Techniques.csv')

In [ ]:
techniques_df.columns

Index(['ID', 'STIX ID', 'name', 'description', 'url', 'created',
       'last modified', 'domain', 'version', 'tactics', 'detection',
       'platforms', 'contributors', 'relationship citations'],
      dtype='object')

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar  3 15:48:25 2024

@author: shahs10
"""

cve_file = "D://family stuff//BERT shahryar Raja//supporting_files//CVE.csv"
technique_file = "D://family stuff//BERT shahryar Raja//supporting_files//Techniques.csv"
cve_embedding_file = "D://family stuff//BERT shahryar Raja//supporting_files//cve_embeddings.csv"
technique_embedding_file = "D://family stuff//BERT shahryar Raja//supporting_files//technique_embeddings.csv"

abbreviation_mapping = {
        'APT': 'Advanced Persistent Threat',
        'AV': 'Antivirus',
        'BOTNET': 'Robot Network',
        'CAPTCHA': 'Automated Public Turing Test to Tell Computers and Humans Apart',
        'CEH': 'Certified Ethical Hacker',
        'CERT': 'Computer Emergency Response Team',
        'CIO': 'Chief Information Officer',
        'COO': 'Chief Operating Officer',
        'CISA': 'Certified Information Systems Auditor',
        'CISA': 'Cybersecurity and Infrastructure Security Agency',
        'CISO': 'Chief Information Security Officer',
        'CISSP': 'Certified Information Systems Security Professional',
        'CMMC': 'Cybersecurity Maturity Model Certification',
        'CSO': 'Chief Security Officer',
        'CSP': 'Cloud Service Provider',
        'CVSS': 'Common Vulnerability Scoring System',
        'DevOps': 'Development and Operations',
        'DevSecOps': 'Development, Security, and Operations',
        'DHS': 'Department of Homeland Security',
        'DoD': 'Department of Defense',
        'DDoS': 'Distributed Denial-of-Service',
        'DoS': 'Denial-of-Service',
        'DLP': 'Data Loss Prevention',
        'DNS': 'Domain Name Server',
        'EDR': 'Endpoint Detection and Response',
        'EO': 'Executive Order',
        'FISMA': 'Federal Information Security Modernization Act',
        'GB': 'Gigabyte',
        'GDPR': 'General Data Protection Regulation',
        'GRC': 'Governance, Risk Management, and Compliance',
        'HIPAA': 'Health Insurance Portability and Accountability Act',
        'HTTPS': 'Hypertext Transfer Protocol Secure',
        'IA': 'Information Assurance'
    }

cve_usable_columns = ['CWE-Description', 'Description', 'Name','Extended Description']
technique_usable_columns = ['name', 'description']

cve_id_column = 'CVE ID'
technique_id_column = 'ID'

model_name = "bert-base-uncased"


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import time
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def read_csv(file_path):
    df = pd.read_csv(file_path)
    return df

def change_abbreviations(df, column_names):
    for column in column_names:
       # Apply the replacement for each row in the column
       df[column] = df[column].apply(lambda x: abbreviation_mapping.get(x, x))

    return df

def combine_columns_cve(df, id_column, concat_columns_list):

    new_df = pd.DataFrame(columns=['ID', 'Combined Description'])
    new_df['Combined Description'] =' '
    for column_name in concat_columns_list:

        new_df['Combined Description'] = df['Description'] + ' ' + df['Name'] + ' ' + df['CWE-Description'] + ' ' + df['Extended Description']
        new_df['Combined Description'] = new_df['Combined Description'].replace("null_value", '')
    new_df['ID'] = df[id_column]

    return new_df

def combine_columns_technique(df, id_column, concat_columns_list):

    new_df = pd.DataFrame(columns=['ID', 'Combined Description'])
    new_df['Combined Description'] =' '
    for column_name in concat_columns_list:

        new_df['Combined Description'] = df['description'] + ' ' + df['name']
        new_df['Combined Description'] = new_df['Combined Description'].replace("null_value", '')
    new_df['ID'] = df[id_column]

    return new_df

def remove_null(df, usable_columns):
    for column in usable_columns:

        # Replace null values with a space
        df[column] = df[column].fillna('null_value')
    return df


def generate_embeddings(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # Extract embeddings from one of the model's output layers
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    return embeddings.numpy().tolist()

def save_embeddings(df, embedding_file_name):
    df.to_csv(embedding_file_name, index=False)


def embedding_generation_phase():
    t1 = time.time()
    print("Started CSV reading")
    cve_df = read_csv(cve_file)
    technique_df = read_csv(technique_file)
    t2 = time.time()
    print("CSV reading completed in {:.2f} seconds".format(t2 - t1))

    t1 = time.time()
    print("Started change_abbreviations")
    cve_df = change_abbreviations(cve_df, cve_usable_columns)
    technique_df = change_abbreviations(technique_df, technique_usable_columns)
    t2 = time.time()
    print("Change abbreviations completed in {:.2f} seconds".format(t2 - t1))

    t1 = time.time()
    print("Started remove_null")
    cve_df = remove_null(cve_df, cve_usable_columns)
    technique_df = remove_null(technique_df, technique_usable_columns)
    t2 = time.time()
    print("Remove null completed in {:.2f} seconds".format(t2 - t1))

    t1 = time.time()
    print("Started combine_columns_cve")
    cve_combined_df = combine_columns_cve(cve_df, cve_id_column, cve_usable_columns)
    technique_combined_df = combine_columns_technique(technique_df, technique_id_column, technique_usable_columns)
    t2 = time.time()
    print("Combine columns completed in {:.2f} seconds".format(t2 - t1))

In [ ]:
technique_df = change_abbreviations(techniques_df, technique_usable_columns)


In [ ]:
    technique_df = remove_null(technique_df, technique_usable_columns)


In [ ]:
    technique_combined_df = combine_columns_technique(technique_df, technique_id_column, technique_usable_columns)


In [ ]:
def generate_embeddings(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, return_dict=True)
    # Extract embeddings from the encoder's last hidden state
    embeddings = torch.mean(outputs.encoder_last_hidden_state, dim=1)
    return embeddings.cpu().numpy().tolist()


In [ ]:
    technique_combined_df['Embedding'] = technique_combined_df['Combined Description'].apply(generate_embeddings)


In [ ]:
technique_combined_df.to_csv('/content/drive/My Drive/shaharyar_bert/finetuned_techniques_embedding.csv', index=False)